### Classification

*Supervised* machine learning techniques involve training a model to operate on a set of *features* and predict a *label* by using a dataset that includes some already-known label values. You can think of this function like this: $y$ represents the label that you want to predict, and $x$ represents the vector of features that the model uses to predict it.

$$y = f([x_1, x_2, x_3, ...])$$

*Classification* is a form of supervised machine learning in which you train a model to use the features (the $x$ values in your function) to predict a label ($y$) that calculates the probability that the observed case belongs to each of a number of possible classes, and to predict an appropriate label. 

### Binary classification

The simplest form of classification is *binary* classification, in which the label is 0 or 1, representing one of two classes. A few examples of binary classification are *True* or *False*; *Internal* or *External*; and *Profitable* or *Non-Profitable*.

In this notebook, you'll focus on an example of binary classification where the model must predict a label that belongs to one of two classes. You'll train a binary classifier to predict whether a patient should be tested for diabetes, based on some medical data.

### Explore the data

The first step in any machine learning project is to explore the data that you'll use to train a model. And before you can explore the data, you must first have it in your environment, right?

Let's begin by importing a CSV file of patient data* into a *tibble* (a modern reimagining of the data frame).

\* The diabetes dataset that we're using in this exercise is based on data originally collected by the National Institute of Diabetes and Digestive and Kidney Diseases.


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)


# Read the CSV file into a tibble
diabetes <- read_csv(file = "https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/data/diabetes.csv")


# Print the first 10 rows of the data
diabetes %>% 
  slice_head(n = 10)


You might occasionally want a little more information about your data. You can view the data, its structure, and the data type of its features by using the [`glimpse()`](https://pillar.r-lib.org/reference/glimpse.html) function, as shown here:



In [ ]:
# Take a quick glance at the data
diabetes %>% 
  glimpse()


This data consists of diagnostic information about some patients who have been tested for diabetes. Note that the final column in the dataset, **Diabetic**, contains the value 0 for patients who tested negative for diabetes, and 1 for patients who tested positive. This is the label that you'll train your model to predict. Most of the other columns, including **Pregnancies**, **PlasmaGlucose**, **DiastolicBloodPressure**, and **BMI**, are the features you'll use to predict the *Diabetic* label.

Let's kick off your adventure by reformatting the data to make it easier for a model to use it effectively. For now, let's drop the **PatientID** column, encode the **Diabetic** column as a categorical variable, and make the column names a bit "friend_lieR," as shown here:


In [ ]:
# Load the janitor package for cleaning data
library(janitor)

# Clean data a bit
diabetes_select <- diabetes %>%
  # Encode Diabetic as category
  mutate(Diabetic = factor(Diabetic, levels = c("1","0"))) %>% 
  # Drop PatientID column
  select(-PatientID) %>% 
  # Clean column names
  clean_names()


# View data set
diabetes_select %>% 
  slice_head(n = 10)


The goal of this exercise is to try to understand the relationships between the data's attributes, in particular, any apparent correlation between the features and the label that your model is trying to predict. One way to do this is to use data visualization.

Now let's compare the feature distributions for each label value. To make it somewhat easier to make multiple *facets*, begin by formatting the data to a *long* format.


In [ ]:
# Pivot data to a long format
diabetes_select_long <- diabetes_select %>% 
    pivot_longer(!diabetic, names_to = "features", values_to = "values")


# Print the first 10 rows
diabetes_select_long %>% 
  slice_head(n = 10)


Perfect! Now, let's make some plots.



In [ ]:
theme_set(theme_light())
# Make a box plot for each predictor feature
diabetes_select_long %>% 
  ggplot(mapping = aes(x = diabetic, y = values, fill = features)) +
  geom_boxplot() + 
  facet_wrap(~ features, scales = "free", ncol = 4) +
  scale_color_viridis_d(option = "plasma", end = .7) +
  theme(legend.position = "none")


Amazing! For some of the features, there's a noticeable difference in the distribution for each label value. In particular, *Pregnancies* and *Age* show markedly different distributions for diabetic patients than for non-diabetic patients. These features can help predict whether a patient is diabetic.

### Split the data

Your dataset includes known values for the label, so you can use them to train a classifier so that it finds a statistical relationship between the features and the label value. But how will you know whether your model is any good? When you use it with new data that it wasn't trained with, how do you know that it will predict correctly?

Well, you can take advantage of the fact that you have a large dataset with known label values. Use only some of it to train the model, and hold back some to test the trained model. This lets you compare the predicted labels with the already-known labels in the test set.

In R, the amazing tidymodels framework provides a collection of packages for modeling and machine learning using *tidyverse* principles. For example, [rsample](https://rsample.tidymodels.org/), a package in tidymodels, provides infrastructure for efficient data splitting and resampling:

-   `initial_split()`: This function specifies how data will be split into a training and testing set.
-   `training()` and `testing()`. These functions extract the data in each split.

Use `?initial_split()` for more details.

> To get started with tidymodels, go to the [Tidymodels](https://www.tidymodels.org/start/) site.


In [ ]:
# Load the tidymodels packages
library(tidymodels)



# Split data into 70% for training and 30% for testing
set.seed(2056)
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.70)


# Extract the data in each split
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)


# Print the number of cases in each split
cat("Training cases: ", nrow(diabetes_train), "\n",
    "Test cases: ", nrow(diabetes_test), sep = "")


# Print out the first 5 rows of the training set
diabetes_train %>% 
  slice_head(n = 5)


### Train and evaluate a binary classification model

Okay, now you're ready to train your model by fitting the training features to the training labels (*Diabetic*). There are various algorithms you can use to train the model. In this example, you'll use *logistic regression*, a well-established algorithm for classification. Logistic regression is a binary classification algorithm, which means that it predicts two categories.

There are many ways to fit a logistic regression model in tidymodels. For more information, see `?logistic_reg()`. For now, let's fit a logistic regression model via the default `stats::glm()` engine.


In [ ]:
# Make a model specifcation
logreg_spec <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


# Print the model specification
logreg_spec


After you've specified a model, you can *estimate* or *train* it by using the [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) function, ordinarily by using a symbolic description of the model (a formula) and some data.



In [ ]:
# Train a logistic regression model
logreg_fit <- logreg_spec %>% 
  fit(diabetic ~ ., data = diabetes_train)


# Print the model object
logreg_fit


The model's printout shows the coefficients learned during the training.

Now that you've trained the model by using the training data, you can use the test data you've held back to evaluate how well it predicts by using [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Start by using the model to predict labels for your test set, and then compare the predicted labels to the known labels:


In [ ]:
# Make predictions then bind them to the test set
results <- diabetes_test %>% select(diabetic) %>% 
  bind_cols(logreg_fit %>% predict(new_data = diabetes_test))


# Compare predictions
results %>% 
  slice_head(n = 10)


Comparing each prediction with its corresponding "ground truth" actual value isn't a very efficient way to determine how well the model is predicting. Fortunately, tidymodels has a few more tricks up its sleeve. You can use the [yardstick package](https://yardstick.tidymodels.org/) to measure the effectiveness of models by using performance metrics.

The most obvious thing you might want to do is check the accuracy of the predictions. In simple terms, what proportion of the labels did the model predict correctly?

The `yardstick::accuracy()` function does that for you.


In [ ]:
# Calculate accuracy: proportion of data predicted correctly
accuracy(data = results, truth = diabetic, estimate = .pred_class)


The level of accuracy is returned as a decimal value. A value of 1.0 would mean that the model got 100 percent of the predictions right. Of course, an accuracy of 0.0 would be pretty useless!

### Summary

In this exercise, you prepared your data by splitting it into test and training datasets. You applied logistic regression, a way of applying binary labels to your data. Your model was able to predict whether patients have diabetes with what appears to be reasonable accuracy. But is "reasonable" accuracy good enough? In the next notebook, you'll look at alternatives to accuracy that can be much more useful in machine learning.
